In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import wandb

api = wandb.Api()


# set theme
sns.set_theme(
    context="paper",
    style="ticks",
    font_scale=0.8,
    rc={
        "figure.figsize": (3.5, 2.0),
        "figure.dpi": 300,
        "savefig.dpi": 300,
        "text.usetex": True,
        "lines.linewidth": 0.7,
        "axes.linewidth": 0.7,
        "axes.grid": True,
        "grid.linestyle": "--",
        "grid.linewidth": 0.5,
        "pdf.fonttype": 42,
    },
)

model_name = "jwtdsmlx"
model_path = Path("../data/test_results") / model_name

# Scalability Experiments
Vary the number of agents and their density. Report the discounted coverage. 

In [ ]:
model_name = "jwtdsmlx"
model_data_dir = Path("../data/test_results/") / model_name

In [ ]:
df = pd.read_parquet(model_data_dir / "scalability.parquet").sort_values(
    ["n_agents", "area"], ascending=False
)
df["density"] = (df["n_agents"] / df["area"]).round(1).apply(lambda x: f"{x:.1f}")
df["time"] = df["step"] * 0.1

## Only vary the number of agents, keep density at 1.0

In [ ]:
# Number of Agents at Density = 1.0 agents per meter squared
plt.figure()
g = sns.lineplot(
    data=(
        df.assign(n_agents=df["n_agents"].apply(lambda x: f"{x:d}")).query(
            "density=='1.0'"
        )
    ),
    x="time",
    y="coverage",
    hue="n_agents",
    palette="rocket_r",
    linewidth=0.8,
    errorbar=None,
)
sns.move_legend(g, "lower right", ncol=1, title="\# of Agents")
plt.ylim(0.50, 1.0)
plt.xlabel("Time")
plt.ylabel("Coverage")
# plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.savefig(model_data_dir / "generalize_scale.pdf", bbox_inches="tight")
plt.show()

In [ ]:
collisions = (
    df.query("density=='1.0'")
    .groupby(["n_agents", "trial"])["collisions"]
    .sum()
    .groupby("n_agents")
    .mean()
)
# normalize to number of agents
collisions / collisions.index * 100

## Vary the density only, n_agents = 100

In [ ]:
# Number of Agents at Density = 1.0 agents per meter squared
plt.figure()
g = sns.lineplot(
    data=df.query("n_agents==100").sort_values("area", ascending=True),
    x="time",
    y="coverage",
    hue="density",
    style="density",
    palette="mako",
    linewidth=0.8,
    errorbar="sd",
)
sns.move_legend(g, "lower right", ncol=1, title="Agent Density")
plt.ylim(0.5, 1.0)
plt.xlabel("Time")
plt.ylabel("Coverage")
# plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.savefig(model_data_dir / "generalize_density.pdf", bbox_inches="tight")
plt.show()

In [ ]:
df.query("n_agents==100").groupby(["density", "trial"])["collisions"].sum().groupby(
    "density"
).mean()

## Now vary both number of agents and density, report coverage

In [ ]:
gamma = 0.99
df["discount"] = gamma ** df["step"]

data: pd.DataFrame = (
    df.groupby(["n_agents", "density", "trial"])
    .apply(
        lambda df: np.average(df["coverage"], weights=df["discount"]),
        include_groups=False,
    )
    .rename("discounted_coverage")  # type: ignore
    .to_frame()
    .sort_values("density", ascending=False)
)


# Number of Agents at Density = 1.0 agents per meter squared
plt.figure()
g = sns.lineplot(
    data=data,
    x="n_agents",
    y="discounted_coverage",
    hue="density",
    style="density",
    markers=".",
    errorbar="sd",
    palette="mako",
)
sns.move_legend(g, "lower right", title="Agent Density")
# plt.ylim(0.80, 0.95)
plt.xlabel("\# of Agents")
plt.ylabel("Average Discounted Coverage")
plt.savefig(model_data_dir / "generalize_scale_density.pdf", bbox_inches="tight")
plt.show()